In [272]:
from web3 import Web3

import time
from datetime import datetime, timedelta
import math
import numpy as np
import pandas as pd
import inspect
import requests
import plotly.express as px
import random
import statsmodels.formula.api as smf
import statsmodels.api as sm

import json
import arrow
import pandas as pd
import datetime as dt
import numpy as np
from decimal import Decimal
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
from elasticsearch_dsl import A, Q

from pycoingecko import CoinGeckoAPI

In [2]:
bearer = <REDACTED>
RPC_URL = f"https://api.anyblock.tools/ethereum/ethereum/mainnet/rpc/{bearer}/"
web3 = Web3(Web3.HTTPProvider(RPC_URL))
es = Elasticsearch(
    hosts=["https://api.anyblock.tools/ethereum/ethereum/mainnet/es/"], 
    headers={"authorization": f"Bearer {bearer}"}, timeout=30, max_retries=10, retry_on_timeout=True)
client = CoinGeckoAPI()

In [3]:
def safeget(dct, *keys):
    """Safely returns dictionary values without throwing out errors and
    without the necessity for try/except

        Parameters
        ----------
        dct : dictionary
        *keys: arbitrary number of keys

        Returns
        -------
        Dict
            value of the final supplied key.
    """
    for key in keys:
        try:
            dct = dct[key]
        except (KeyError, TypeError):
            return dict()
    return dct

In [4]:
nouns_treasury = '0x0BC3807Ec262cB779b38D65b38158acC3bfedE10'
nouns_treasury_abi = json.loads("""[{"inputs":[{"internalType":"address","name":"admin_","type":"address"},{"internalType":"uint256","name":"delay_","type":"uint256"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"txHash","type":"bytes32"},{"indexed":true,"internalType":"address","name":"target","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"string","name":"signature","type":"string"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"},{"indexed":false,"internalType":"uint256","name":"eta","type":"uint256"}],"name":"CancelTransaction","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"txHash","type":"bytes32"},{"indexed":true,"internalType":"address","name":"target","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"string","name":"signature","type":"string"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"},{"indexed":false,"internalType":"uint256","name":"eta","type":"uint256"}],"name":"ExecuteTransaction","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newAdmin","type":"address"}],"name":"NewAdmin","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"newDelay","type":"uint256"}],"name":"NewDelay","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"newPendingAdmin","type":"address"}],"name":"NewPendingAdmin","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"bytes32","name":"txHash","type":"bytes32"},{"indexed":true,"internalType":"address","name":"target","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"},{"indexed":false,"internalType":"string","name":"signature","type":"string"},{"indexed":false,"internalType":"bytes","name":"data","type":"bytes"},{"indexed":false,"internalType":"uint256","name":"eta","type":"uint256"}],"name":"QueueTransaction","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"GRACE_PERIOD","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MAXIMUM_DELAY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"MINIMUM_DELAY","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"acceptAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"admin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"string","name":"signature","type":"string"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"uint256","name":"eta","type":"uint256"}],"name":"cancelTransaction","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"delay","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"string","name":"signature","type":"string"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"uint256","name":"eta","type":"uint256"}],"name":"executeTransaction","outputs":[{"internalType":"bytes","name":"","type":"bytes"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"pendingAdmin","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"target","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"},{"internalType":"string","name":"signature","type":"string"},{"internalType":"bytes","name":"data","type":"bytes"},{"internalType":"uint256","name":"eta","type":"uint256"}],"name":"queueTransaction","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"name":"queuedTransactions","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"delay_","type":"uint256"}],"name":"setDelay","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"pendingAdmin_","type":"address"}],"name":"setPendingAdmin","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]""")
convex_treasury = '0x1389388d01708118b497f59521f6943Be2541bb7'
erc20_abi = json.loads("""[{"inputs":[{"internalType":"address","name":"_proxy","type":"address"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"account","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"maxSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"mint","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"operator","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"reductionPerCliff","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalCliffs","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"sender","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"updateOperator","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"vecrvProxy","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"}]""")
chainlink_convex = '0x9956735F34011f2EedF91a5c1f704ae1254C8010'
chainlink_abi = json.loads("""[{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"},{"internalType":"contract LinkTokenInterface","name":"_link","type":"address"},{"internalType":"int192","name":"_minAnswer","type":"int192"},{"internalType":"int192","name":"_maxAnswer","type":"int192"},{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"},{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"},{"internalType":"uint8","name":"_decimals","type":"uint8"},{"internalType":"string","name":"description","type":"string"}],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"AddedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"int256","name":"current","type":"int256"},{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"updatedAt","type":"uint256"}],"name":"AnswerUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"BillingAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"indexed":false,"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"name":"BillingSet","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessDisabled","type":"event"},{"anonymous":false,"inputs":[],"name":"CheckAccessEnabled","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"uint32","name":"previousConfigBlockNumber","type":"uint32"},{"indexed":false,"internalType":"uint64","name":"configCount","type":"uint64"},{"indexed":false,"internalType":"address[]","name":"signers","type":"address[]"},{"indexed":false,"internalType":"address[]","name":"transmitters","type":"address[]"},{"indexed":false,"internalType":"uint8","name":"threshold","type":"uint8"},{"indexed":false,"internalType":"uint64","name":"encodedConfigVersion","type":"uint64"},{"indexed":false,"internalType":"bytes","name":"encoded","type":"bytes"}],"name":"ConfigSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"contract LinkTokenInterface","name":"_oldLinkToken","type":"address"},{"indexed":true,"internalType":"contract LinkTokenInterface","name":"_newLinkToken","type":"address"}],"name":"LinkTokenSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"roundId","type":"uint256"},{"indexed":true,"internalType":"address","name":"startedBy","type":"address"},{"indexed":false,"internalType":"uint256","name":"startedAt","type":"uint256"}],"name":"NewRound","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint32","name":"aggregatorRoundId","type":"uint32"},{"indexed":false,"internalType":"int192","name":"answer","type":"int192"},{"indexed":false,"internalType":"address","name":"transmitter","type":"address"},{"indexed":false,"internalType":"int192[]","name":"observations","type":"int192[]"},{"indexed":false,"internalType":"bytes","name":"observers","type":"bytes"},{"indexed":false,"internalType":"bytes32","name":"rawReportContext","type":"bytes32"}],"name":"NewTransmission","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"payee","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":true,"internalType":"contract LinkTokenInterface","name":"linkToken","type":"address"}],"name":"OraclePaid","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"}],"name":"OwnershipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"},{"indexed":true,"internalType":"address","name":"proposed","type":"address"}],"name":"PayeeshipTransferRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"transmitter","type":"address"},{"indexed":true,"internalType":"address","name":"previous","type":"address"},{"indexed":true,"internalType":"address","name":"current","type":"address"}],"name":"PayeeshipTransferred","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"user","type":"address"}],"name":"RemovedAccess","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"contract AccessControllerInterface","name":"old","type":"address"},{"indexed":false,"internalType":"contract AccessControllerInterface","name":"current","type":"address"}],"name":"RequesterAccessControllerSet","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"requester","type":"address"},{"indexed":false,"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"indexed":false,"internalType":"uint32","name":"epoch","type":"uint32"},{"indexed":false,"internalType":"uint8","name":"round","type":"uint8"}],"name":"RoundRequested","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"contract AggregatorValidatorInterface","name":"previousValidator","type":"address"},{"indexed":false,"internalType":"uint32","name":"previousGasLimit","type":"uint32"},{"indexed":true,"internalType":"contract AggregatorValidatorInterface","name":"currentValidator","type":"address"},{"indexed":false,"internalType":"uint32","name":"currentGasLimit","type":"uint32"}],"name":"ValidatorConfigSet","type":"event"},{"inputs":[],"name":"acceptOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"acceptPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"addAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"billingAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"checkEnabled","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"description","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"disableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"enableAccessCheck","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBilling","outputs":[{"internalType":"uint32","name":"maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"linkGweiPerTransmission","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getLinkToken","outputs":[{"internalType":"contract LinkTokenInterface","name":"linkToken","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint80","name":"_roundId","type":"uint80"}],"name":"getRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_roundId","type":"uint256"}],"name":"getTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"},{"internalType":"bytes","name":"_calldata","type":"bytes"}],"name":"hasAccess","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestAnswer","outputs":[{"internalType":"int256","name":"","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestConfigDetails","outputs":[{"internalType":"uint32","name":"configCount","type":"uint32"},{"internalType":"uint32","name":"blockNumber","type":"uint32"},{"internalType":"bytes16","name":"configDigest","type":"bytes16"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRound","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestRoundData","outputs":[{"internalType":"uint80","name":"roundId","type":"uint80"},{"internalType":"int256","name":"answer","type":"int256"},{"internalType":"uint256","name":"startedAt","type":"uint256"},{"internalType":"uint256","name":"updatedAt","type":"uint256"},{"internalType":"uint80","name":"answeredInRound","type":"uint80"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTimestamp","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"latestTransmissionDetails","outputs":[{"internalType":"bytes16","name":"configDigest","type":"bytes16"},{"internalType":"uint32","name":"epoch","type":"uint32"},{"internalType":"uint8","name":"round","type":"uint8"},{"internalType":"int192","name":"latestAnswer","type":"int192"},{"internalType":"uint64","name":"latestTimestamp","type":"uint64"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"linkAvailableForPayment","outputs":[{"internalType":"int256","name":"availableBalance","type":"int256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"maxAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"minAnswer","outputs":[{"internalType":"int192","name":"","type":"int192"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_signerOrTransmitter","type":"address"}],"name":"oracleObservationCount","outputs":[{"internalType":"uint16","name":"","type":"uint16"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"owedPayment","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address payable","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_user","type":"address"}],"name":"removeAccess","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requestNewRound","outputs":[{"internalType":"uint80","name":"","type":"uint80"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"requesterAccessController","outputs":[{"internalType":"contract AccessControllerInterface","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint32","name":"_maximumGasPrice","type":"uint32"},{"internalType":"uint32","name":"_reasonableGasPrice","type":"uint32"},{"internalType":"uint32","name":"_microLinkPerEth","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerObservation","type":"uint32"},{"internalType":"uint32","name":"_linkGweiPerTransmission","type":"uint32"}],"name":"setBilling","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_billingAccessController","type":"address"}],"name":"setBillingAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_signers","type":"address[]"},{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"uint8","name":"_threshold","type":"uint8"},{"internalType":"uint64","name":"_encodedConfigVersion","type":"uint64"},{"internalType":"bytes","name":"_encoded","type":"bytes"}],"name":"setConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract LinkTokenInterface","name":"_linkToken","type":"address"},{"internalType":"address","name":"_recipient","type":"address"}],"name":"setLinkToken","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address[]","name":"_transmitters","type":"address[]"},{"internalType":"address[]","name":"_payees","type":"address[]"}],"name":"setPayees","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AccessControllerInterface","name":"_requesterAccessController","type":"address"}],"name":"setRequesterAccessController","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"contract AggregatorValidatorInterface","name":"_newValidator","type":"address"},{"internalType":"uint32","name":"_newGasLimit","type":"uint32"}],"name":"setValidatorConfig","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_to","type":"address"}],"name":"transferOwnership","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"},{"internalType":"address","name":"_proposed","type":"address"}],"name":"transferPayeeship","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"bytes","name":"_report","type":"bytes"},{"internalType":"bytes32[]","name":"_rs","type":"bytes32[]"},{"internalType":"bytes32[]","name":"_ss","type":"bytes32[]"},{"internalType":"bytes32","name":"_rawVs","type":"bytes32"}],"name":"transmit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"transmitters","outputs":[{"internalType":"address[]","name":"","type":"address[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"typeAndVersion","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"pure","type":"function"},{"inputs":[],"name":"validatorConfig","outputs":[{"internalType":"contract AggregatorValidatorInterface","name":"validator","type":"address"},{"internalType":"uint32","name":"gasLimit","type":"uint32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"version","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_recipient","type":"address"},{"internalType":"uint256","name":"_amount","type":"uint256"}],"name":"withdrawFunds","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"_transmitter","type":"address"}],"name":"withdrawPayment","outputs":[],"stateMutability":"nonpayable","type":"function"}]""")


In [6]:
def fetch_internal_transactions(treasury_contract):
    data = []
    labels = {
        'to': 'revenue',
        'from': 'expense',
    }
    for direction in ('to', 'from'):
        q = {
            "query": {
                "bool": {
                    "filter": [
                        {
                            "nested": {
                                "path": "action",
                                "query": {
                                    "bool": {
                                        "filter": [
                                            {
                                                "term": {
                                                    f"action.{direction}.raw": treasury_contract
                                                }
                                            }
                                        ]
                                    }
                                }
                            }
                        }
                    ]
                }
            },
            "sort": [
                {
                    "blockNumber.num": "desc"
                }
            ],
            "size": 1000
        }

        s = Search(using=es, index="trace") \
            .update_from_dict(q)
        for hit in s.scan():
            data.append({
                'timestamp': safeget(hit, 'timestamp'),
                "datetime": pd.to_datetime(safeget(hit, 'timestamp'), unit='s'),
                "block_number": safeget(hit, 'blockNumber', 'num'),
                "direction": direction,
                "type": safeget(hit, 'type'),
                'from': safeget(hit, 'action', 'from'),
                'to': safeget(hit, 'action', 'to'),
                labels[direction]: int(safeget(hit, 'action', 'value', 'raw')) / 1e18,
            })
    
    
    df = pd.DataFrame(data)
    df = df.sort_values('timestamp', ascending=True).reset_index(drop=True)
    return df

In [7]:
df_internal = fetch_internal_transactions(nouns_treasury)

In [8]:
df_internal

,timestamp,datetime,block_number,direction,type,from,to,revenue,expense
0,1628531228,2021-08-09 17:47:08,12992388,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,613.3700,NaN
1,1628617690,2021-08-10 17:48:10,12998889,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,69.6900,NaN
2,1628705130,2021-08-11 18:05:30,13005388,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,36.6900,NaN
3,1628792090,2021-08-12 18:14:50,13011907,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,45.0000,NaN
4,1628878904,2021-08-13 18:21:44,13018438,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,50.0000,NaN
...,...,...,...,...,...,...,...,...,...
1320,1660948358,2022-08-19 22:32:38,15373929,to,call,0xfAa4bbe589a39745833e2BecE8d401b6195A07b1,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,0.0250,NaN
1321,1660977170,2022-08-20 06:32:50,15376070,to,call,0x830BD73E4184ceF73443C15111a1DF14e495C706,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,61.2000,NaN
1322,1660977677,2022-08-20 06:41:17,15376108,to,call,0xfAa4bbe589a39745833e2BecE8d401b6195A07b1,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,0.0275,NaN
1323,1661006807,2022-08-20 14:46:47,15378220,to,call,0xfAa4bbe589a39745833e2BecE8d401b6195A07b1,0x0BC3807Ec262cB779b38D65b38158acC3bfedE10,0.0175,NaN


In [11]:
df_internal.index = df_internal['datetime']

In [159]:
df_internal_sampled = df_internal.resample('M').sum()

In [160]:
# Graphing the data
fig = px.bar(df_internal_sampled, x = df_internal_sampled.index, y=['revenue', 'expense'],
    height=500, title = "Nouns Cash Flows (ETH)")
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x", yaxis= {"tickformat": 'Ξ' })
fig.show() 

In [54]:
def fetch_balances(treasury):
    q = {
        "query": {
            "bool": {
                "must": [
                    {
                        "term": {
                            "address.raw": treasury
                        }
                    }
                ]
            }
        },
        "sort": [
            {
                "blockNumber.num": "asc"
            }
        ],
        "size": 1000
    }
    
    s = Search(using=es, index="balance") \
        .update_from_dict(q)
    data = []
    for hit in s.scan():
        data.append({
            "timestamp": pd.to_datetime(safeget(hit, 'timestamp'), unit='s'),
            "block_number": safeget(hit, 'blockNumber', 'num'),
            "balance": safeget(hit, 'balance', 'eth')
        })
    
    
    df = pd.DataFrame(data)
    df = df.sort_values('timestamp', ascending=True).reset_index(drop=True)
    return df

In [66]:
df_balances_nouns = fetch_balances(nouns_treasury)

In [158]:
df_balances_nouns.index = df_balances_nouns['timestamp']
df_balances_nouns = df_balances_nouns.resample('M').max()

In [68]:
df_balances_nouns

,timestamp,block_number,balance
timestamp,,,
2021-08-08,2021-08-08 16:04:46,12985452.0,0.000000
2021-08-09,2021-08-09 17:47:08,12992388.0,613.370000
2021-08-10,2021-08-10 17:48:10,12998889.0,683.060000
2021-08-11,2021-08-11 18:05:30,13005388.0,719.750000
2021-08-12,2021-08-12 18:14:50,13011907.0,764.750000
...,...,...,...
2022-08-16,2022-08-16 21:53:46,15354868.0,22588.734181
2022-08-17,2022-08-17 19:28:02,15360556.0,22665.049181
2022-08-18,2022-08-18 22:03:21,15367524.0,22441.789181


In [161]:
df_nouns = pd.merge(df_balances_nouns[['balance']], df_internal_sampled[['revenue', 'expense']], left_index = True, right_index=True)

In [206]:
# Create some faux debt
df_nouns['Equity'] = df_nouns['balance']
df_nouns['equity_diff'] = df_nouns['Equity'].diff()
df_nouns['debt_diff'] = df_nouns['equity_diff'].apply(lambda x: x * .8 * np.random.rand())
df_nouns['Debt'] = df_nouns['debt_diff'].cumsum()
df_nouns.iloc[0,4] = 0
df_nouns['Total value'] = df_nouns['Equity'] + df_nouns['Debt']
df_nouns['Debt/Equity ratio'] = df_nouns['Debt'] / df_nouns['Equity']

In [207]:
df_nouns

,balance,revenue,expense,Equity,equity_diff,debt_diff,Debt,Total value,Debt/Equity ratio
timestamp,,,,,,,,,
2021-08-31,3455.602200,3485.602200,30.00000,3455.602200,0.000000,NaN,NaN,NaN,NaN
2021-09-30,7700.409200,4487.807000,243.00000,7700.409200,4244.807000,1958.436770,1958.436770,9658.845970,0.254329
2021-10-31,13291.743200,6067.470000,116.13600,13291.743200,5591.334000,2078.263364,4036.700135,17328.443335,0.303700
2021-11-30,15608.733200,2428.200000,139.21000,15608.733200,2316.990000,1849.754918,5886.455052,21495.188252,0.377126
2021-12-31,15734.753200,1443.216000,1707.15000,15734.753200,126.020000,18.181927,5904.636979,21639.390179,0.375261
2022-01-31,15909.919200,2274.050000,1639.20000,15909.919200,175.166000,62.263061,5966.900040,21876.819240,0.375043
2022-02-28,17796.905060,2287.630000,701.62414,17796.905060,1886.985860,1090.003034,7056.903075,24853.808135,0.396524
2022-03-31,18998.169560,1918.864500,479.14414,18998.169560,1201.264500,803.891485,7860.794560,26858.964120,0.413766
2022-04-30,19732.635420,2035.120000,2529.21140,19732.635420,734.465860,140.429867,8001.224427,27733.859847,0.405482


In [208]:
# Graphing the data
fig = px.line(df_nouns, x = df_nouns.index, y=['Debt/Equity ratio'],
              height=500, title = "Nouns DAO Financial Statement",
              labels={
                     "value": "ETH",
                     "index": "Date",
                 },
             )
fig.update_layout(hovermode="x", yaxis= {"tickformat": '.2%' })
fig.show() 

In [219]:
# Graphing the data
fig = px.line(df_nouns, x = df_nouns.index, y=['Total value', 'Equity', 'Debt', 'revenue', 'expense'],
              height=500, title = "Nouns DAO Financial Statement",
              labels={
                     "value": "ETH",
                     "index": "Date",
                 },
             )
fig.update_layout(hovermode="x", yaxis= {'ticksuffix': ' ETH' })
fig.show() 

In [211]:
df_nouns

,balance,revenue,expense,Equity,equity_diff,debt_diff,Debt,Total value,Debt/Equity ratio
timestamp,,,,,,,,,
2021-08-31,3455.602200,3485.602200,30.00000,3455.602200,0.000000,NaN,NaN,NaN,NaN
2021-09-30,7700.409200,4487.807000,243.00000,7700.409200,4244.807000,1958.436770,1958.436770,9658.845970,0.254329
2021-10-31,13291.743200,6067.470000,116.13600,13291.743200,5591.334000,2078.263364,4036.700135,17328.443335,0.303700
2021-11-30,15608.733200,2428.200000,139.21000,15608.733200,2316.990000,1849.754918,5886.455052,21495.188252,0.377126
2021-12-31,15734.753200,1443.216000,1707.15000,15734.753200,126.020000,18.181927,5904.636979,21639.390179,0.375261
2022-01-31,15909.919200,2274.050000,1639.20000,15909.919200,175.166000,62.263061,5966.900040,21876.819240,0.375043
2022-02-28,17796.905060,2287.630000,701.62414,17796.905060,1886.985860,1090.003034,7056.903075,24853.808135,0.396524
2022-03-31,18998.169560,1918.864500,479.14414,18998.169560,1201.264500,803.891485,7860.794560,26858.964120,0.413766
2022-04-30,19732.635420,2035.120000,2529.21140,19732.635420,734.465860,140.429867,8001.224427,27733.859847,0.405482


In [306]:
df_nouns.tail(5)['expense'].pct_change().mul(1).rolling(4).mean()

timestamp
2022-04-30         NaN
2022-05-31         NaN
2022-06-30         NaN
2022-07-31         NaN
2022-08-31    0.642117
Name: expense, dtype: float64

In [270]:
df_nouns.tail(5)['revenue'].pct_change().mul(1).rolling(4).mean()

timestamp
2022-04-30         NaN
2022-05-31         NaN
2022-06-30         NaN
2022-07-31         NaN
2022-08-31    0.070756
Name: revenue, dtype: float64

In [313]:
df_nouns.tail(5)['expense'].pct_change().mul(1).rolling(4).mean()[-1]

0.642116899769227

In [325]:
def randomwalk1D(n, cutoff):
    x, y = df_nouns.index[-1], df_nouns.iloc[-1, 3]
    monthly_move = (df_nouns.iloc[-1, 3] - df_nouns.iloc[-13, 3])/12
    expense = df_nouns.tail(10)['expense'].mean()
    revenue = df_nouns.tail(10)['revenue'].mean()
    expense_move = df_nouns.tail(5)['expense'].pct_change().mul(1).rolling(4).mean()[-1]
    revenue_move = df_nouns.tail(5)['revenue'].pct_change().mul(1).rolling(4).mean()[-1]

    # Generate the time points [1, 2, 3, ... , n]
    points = np.arange(n + 1)
    timepoints = []
    for timepoint in points:
        timepoint = x + timedelta(days=int(timepoint))
        timepoints.append(timepoint)
    positions = [y]

    directions = ["UP", "DOWN"]
    for i in range(1, n + 1):

        # Randomly select either UP or DOWN
        step = random.choice(directions)
        
        # Move the object up or down
        if step == "UP":
            y += (revenue * i * .5 * revenue_move - expense)
        elif step == "DOWN":
            y += (revenue - expense * i * .5 * expense_move)

        # Keep track of the positions
        positions.append(y)

#     return timepoints, positions

    if y > cutoff:
        return 1
    else:
        return 0

In [326]:
results = []
for reps in range(0, 10000):
    results.append(randomwalk1D(12, df_nouns.iloc[-1, 6]))

In [328]:
# Chance of default
np.mean(results)

0.9179

# Unused code

### Convex

In [14]:
def fetch_token_transactions(treasury_contract):
    data = []
    labels = {
        'to': 'revenue',
        'from': 'expense',
    }
    q = {
        "query": {
            "bool": {
                "filter": [
                    {
                        "term": {
                            "event.raw": "Transfer"
                        }
                    },
                    {
                        "nested": {
                            "path": "args",
                            "query": {
                                "bool": {
                                    "filter": [
                                        {
                                            "term": {
                                                "args.value.hex": treasury_contract
                                            }
                                        }
                                    ]
                                }
                            }
                        }
                    }
                ]
            }
        },
        "size": 1000
    }

    s = Search(using=es, index="event") \
        .update_from_dict(q)
    for hit in s.scan():
        for arg in safeget(hit, 'args'):
            if arg['name'] == 'from':
                tx_from = arg['value.hex']
            if arg['name'] == 'to':
                tx_to = arg['value.hex']
            if arg['name'] == 'value':
                value_hex = arg['value.hex']
                value_scaled = arg['value.scaled']
        if tx_to == treasury_contract:
            direction = 'to'
        else:
            direction = 'from'
        data.append({
            "timestamp": safeget(hit, 'timestamp'),
            'datetime': pd.to_datetime(safeget(hit, 'timestamp'), unit='s'),
            "block_number": safeget(hit, 'blockNumber', 'num'),
            "direction": direction,
            "type": safeget(hit, 'event'),
            'token': safeget(hit, 'address'),
            'from': tx_from,
            'to': tx_to,
            labels[direction]: value_scaled,
            labels[direction]+'_hex': value_hex,
        })
    
    
    df = pd.DataFrame(data)
    df = df.sort_values('timestamp', ascending=True).reset_index(drop=True)
    return df

In [15]:
df_convex = fetch_token_transactions(convex_treasury)

In [16]:
df_convex

,timestamp,datetime,block_number,direction,type,token,from,to,revenue,revenue_hex,expense,expense_hex
0,1621820223,2021-05-24 01:37:03,12494128,to,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xe98984aD858075813AdA4261aF47e68A64E28fCC,0x1389388d01708118b497f59521f6943Be2541bb7,176006.502467,0x254554e4dfb870cd8c5f,NaN,NaN
1,1622620852,2021-06-02 08:00:52,12553800,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,90000.0,0x130ee8e7179044400000
2,1623439046,2021-06-11 19:17:26,12615075,to,Transfer,0x7DDdcb8D93438500c42c199b6B02ba3F74C7cB8c,0x4172Dc63CB0ECB9FB8a224D565Be8b80d2a1a14f,0x1389388d01708118b497f59521f6943Be2541bb7,290000.000000,0x3d68ee76da2631400000,NaN,NaN
3,1623722468,2021-06-15 02:01:08,12636238,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x1ca46B24A28Edb718f85cE5DEd01Ccaf578ED666,NaN,NaN,100.0,0x56bc75e2d63100000
4,1630508488,2021-09-01 15:01:28,13140460,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,7000.0,0x17b7883c06916600000
...,...,...,...,...,...,...,...,...,...,...,...,...
64,1655952028,2022-06-23 02:40:28,15010781,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000
65,1656212593,2022-06-26 03:03:13,15026885,to,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x5e2706946c2bE55b038f4a0475cb7b19f5a67897,0x1389388d01708118b497f59521f6943Be2541bb7,736835.045662,0x9c07e869be4b448cca00,NaN,NaN
66,1657154752,2022-07-07 00:45:52,15092290,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000
67,1658370924,2022-07-21 02:35:24,15183288,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000


In [22]:
price_data = {}
for token in df_convex['token'].unique():
    price_data[token] = {}
    try:
        result = client.get_coin_market_chart_range_from_contract_address_by_id("ethereum", 
                                                               token, 
                                                               "ETH",               
                                                               df_convex.iloc[0, 0], 
                                                               df_convex.iloc[-1, 0])
        prices = result['prices']
        price_data[token]['prices'] = {}
        for price in prices:
            price_data[token]['prices'][price[0]] = price[1]
        mcaps = result['market_caps']
        price_data[token]['mcaps'] = {}
        for mcap in mcaps:
            price_data[token]['mcaps'][mcap[0]] = mcap[1]
        print(f"Found token {token}")
    except ValueError:
        print(f"Could not find token {token}")
    time.sleep(5)

Found token 0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B
Could not find token 0x7DDdcb8D93438500c42c199b6B02ba3F74C7cB8c
Found token 0xdB25f211AB05b1c97D595516F45794528a807ad8
Found token 0xa3BeD4E1c75D00fa6f4E5E6922DB7261B5E9AcD2
Found token 0xdBdb4d16EdA451D0503b854CF79D55697F90c8DF
Found token 0x3472A5A71965499acd81997a54BBA8D852C6E53d
Found token 0x4E15361FD6b4BB609Fa63C81A2be19d873717870
Found token 0x8207c1FfC5B6804F6024322CcF34F29c3541Ae26
Found token 0xD533a949740bb3306d119CC777fa900bA034cd52
Found token 0x31429d1856aD1377A8A0079410B297e1a9e214c2
Found token 0xCdF7028ceAB81fA0C6971208e83fa7872994beE5
Could not find token 0x9C80f6719018262EDF780D0158A770ab99EDe4F8
Could not find token 0x4Eb8b4C65D8430647586cf44af4Bf23dEd2Bb794
Found token 0x92E187a03B6CD19CB6AF293ba17F2745Fd2357D5
Found token 0x5A98FcBEA516Cf06857215779Fd812CA3beF1B32
Found token 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0
Could not find token 0x4a41775Da459B38e641141e4C696DF10EC1f4983
Could not find token 0xd9C082

In [23]:
def convert_token_to_eth(df):
    revenue_eth_col = []
    expense_eth_col = []
    prices_col = []
    mcap_col = []
    for i, row in df.iterrows():
        print(i, ' of ', len(df))
        block_identifier = row['block_number']
        token = row['token']
        timestamp = row['timestamp']
        contract_token = web3.eth.contract(address=token, abi=erc20_abi)
        decimals = contract_token.functions.decimals().call(block_identifier=block_identifier)
        
        try:
            prices = price_data[token]['prices']
            mcaps = price_data[token]['mcaps']
            price = prices[min(prices, key=lambda x:abs(x-timestamp*1000))]
            mcap = mcaps[min(mcaps, key=lambda x:abs(x-timestamp*1000))]
        except KeyError:
            print(f"No price data for token {token}")
            price = np.nan
            mcap = np.nan
        
        prices_col.append(price)
        mcap_col.append(mcap)
        revenue_eth_col.append(row['revenue'] / decimals * price)
        expense_eth_col.append(row['expense'] / decimals * price)
        time.sleep(1)
    df['prices'] = prices_col
    df['mcap'] = mcap_col
    df['revenue_eth'] = revenue_eth_col
    df['expense_eth'] = expense_eth_col
    return df

In [24]:
# def convert_token_to_eth(df, chainlink_feed):
#     revenue_eth = []
#     expense_eth = []
#     cvx_eth_price = []
#     for i, row in df.iterrows():
#         print(i, ' of ', len(df))
#         block_identifier = row['block_number']
#         contract_token = web3.eth.contract(address=row['token'], abi=erc20_abi)
#         decimals = contract_token.functions.decimals().call(block_identifier=block_identifier)
        
#         contract_chainlink_feed = web3.eth.contract(address=chainlink_feed, abi=chainlink_abi)
#         try:
#             price = contract_chainlink_feed.functions.latestAnswer().call(block_identifier=block_identifier)
#         except Exception as e:
#             price = 0
#         cvx_eth_price.append(price)
#         revenue_eth.append(row['revenue'] / decimals * (price/1e18))
#         expense_eth.append(row['expense'] / decimals * (price/1e18))
#         time.sleep(3)
#     df['revenue_eth'] = revenue_eth
#     df['expense_eth'] = expense_eth
#     return df

In [25]:
df_convex = convert_token_to_eth(df_convex)

0  of  69
1  of  69
2  of  69
No price data for token 0x7DDdcb8D93438500c42c199b6B02ba3F74C7cB8c
3  of  69
4  of  69
5  of  69
6  of  69
7  of  69
8  of  69
9  of  69
10  of  69
11  of  69
12  of  69
13  of  69
14  of  69
15  of  69
16  of  69
17  of  69
18  of  69
19  of  69
20  of  69
21  of  69
22  of  69
23  of  69
24  of  69
25  of  69
26  of  69
27  of  69
28  of  69
29  of  69
30  of  69
31  of  69
32  of  69
33  of  69
No price data for token 0x9C80f6719018262EDF780D0158A770ab99EDe4F8
34  of  69
No price data for token 0x9C80f6719018262EDF780D0158A770ab99EDe4F8
35  of  69
36  of  69
37  of  69
38  of  69
No price data for token 0x4Eb8b4C65D8430647586cf44af4Bf23dEd2Bb794
39  of  69
40  of  69
41  of  69
42  of  69
43  of  69
44  of  69
45  of  69
46  of  69
47  of  69
48  of  69
49  of  69
50  of  69
51  of  69
52  of  69
No price data for token 0x4a41775Da459B38e641141e4C696DF10EC1f4983
53  of  69
No price data for token 0x4a41775Da459B38e641141e4C696DF10EC1f4983
54  of  69
No 

In [26]:
df_convex

,timestamp,datetime,block_number,direction,type,token,from,to,revenue,revenue_hex,expense,expense_hex,prices,mcap,revenue_eth,expense_eth
0,1621820223,2021-05-24 01:37:03,12494128,to,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0xe98984aD858075813AdA4261aF47e68A64E28fCC,0x1389388d01708118b497f59521f6943Be2541bb7,176006.502467,0x254554e4dfb870cd8c5f,NaN,NaN,0.002424,4312.216550,23.705533,NaN
1,1622620852,2021-06-02 08:00:52,12553800,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,90000.0,0x130ee8e7179044400000,0.003654,8867.467500,NaN,18.270023
2,1623439046,2021-06-11 19:17:26,12615075,to,Transfer,0x7DDdcb8D93438500c42c199b6B02ba3F74C7cB8c,0x4172Dc63CB0ECB9FB8a224D565Be8b80d2a1a14f,0x1389388d01708118b497f59521f6943Be2541bb7,290000.000000,0x3d68ee76da2631400000,NaN,NaN,NaN,NaN,NaN,NaN
3,1623722468,2021-06-15 02:01:08,12636238,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x1ca46B24A28Edb718f85cE5DEd01Ccaf578ED666,NaN,NaN,100.0,0x56bc75e2d63100000,0.002267,10787.964525,NaN,0.012595
4,1630508488,2021-09-01 15:01:28,13140460,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,7000.0,0x17b7883c06916600000,0.001889,40793.804863,NaN,0.734783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1655952028,2022-06-23 02:40:28,15010781,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000,0.004012,255575.798586,NaN,44.575222
65,1656212593,2022-06-26 03:03:13,15026885,to,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x5e2706946c2bE55b038f4a0475cb7b19f5a67897,0x1389388d01708118b497f59521f6943Be2541bb7,736835.045662,0x9c07e869be4b448cca00,NaN,NaN,0.003938,251314.551197,161.202638,NaN
66,1657154752,2022-07-07 00:45:52,15092290,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000,0.005006,328922.206765,NaN,55.620665
67,1658370924,2022-07-21 02:35:24,15183288,from,Transfer,0x4e3FBD56CD56c3e72c1403e103b45Db9da5B9D2B,0x1389388d01708118b497f59521f6943Be2541bb7,0x947B7742C403f20e5FaCcDAc5E092C943E7D0277,NaN,NaN,200000.0,0x2a5a058fc295ed000000,0.004774,313818.946419,NaN,53.045798


In [27]:
df_convex.index = df_convex['datetime']
df_convex_sampled = df_convex.resample('m').sum()

In [28]:
# Graphing the data
fig = px.bar(df_convex_sampled, x = df_convex_sampled.index, y=['revenue_eth', 'expense_eth'],
    height=500, title = "Convex DAO Cash Flows (ETH)")
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x", yaxis= {"tickformat": 'Ξ' })
fig.show()

In [34]:
def get_graph_query(subgraph_url, query, variables=None):
    request_json = {'query': query}
    if variables:
        request_json['variables'] = variables
    resp = requests.post(subgraph_url, json=request_json)
    response = json.loads(resp.text)
    response = response['data']

    return response

In [35]:
query = """query MyQuery {
  financialsDailySnapshots {
    cumulativeProtocolSideRevenueUSD
    cumulativeSupplySideRevenueUSD
    timestamp
    id
    blockNumber
    cumulativeTotalRevenueUSD
    dailyProtocolSideRevenueUSD
    dailySupplySideRevenueUSD
    dailyTotalRevenueUSD
    protocolControlledValueUSD
    totalValueLockedUSD
  }
}"""
subgraph_convex = 'https://api.thegraph.com/subgraphs/name/messari/convex-finance-ethereum'
df = pd.DataFrame(get_graph_query(subgraph_convex, query)['financialsDailySnapshots'])

In [38]:
df.index = pd.to_datetime(df['timestamp'], unit='s')

In [39]:
df

,cumulativeProtocolSideRevenueUSD,cumulativeSupplySideRevenueUSD,timestamp,id,blockNumber,cumulativeTotalRevenueUSD,dailyProtocolSideRevenueUSD,dailySupplySideRevenueUSD,dailyTotalRevenueUSD,protocolControlledValueUSD,totalValueLockedUSD
timestamp,,,,,,,,,,,
2021-05-17 23:55:37,0,0,1621295737,18764,12455009,0,0,0,0,0,60676928.74190330106055586944512512
2021-05-18 23:58:29,2911.5,104793.3,1621382309,18765,12461481,107704.8,2911.5,104793.3,107704.8,0,167517064.3852805606505050426576255
2021-05-19 23:57:47,2911.5,104793.3,1621468667,18766,12467900,107704.8,0,0,0,0,139099538.6602314430271684345698088
2021-05-20 23:57:13,13123.9,501963.6,1621555033,18767,12474361,515087.5,10212.4,397170.3,407382.7,0,160690917.0157650690238170354105772
2021-05-21 23:47:32,19545.1,746543.1,1621640852,18768,12480724,766088.2,6421.2,244579.5,251000.7,0,196388866.954201839408676529546327
...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 23:57:59,3898320.5,126508701.6,1629503879,18859,13065312,130407022.1,14404.7,497599,512003.7,0,6160465564.22638174839381445340781
2021-08-21 23:52:04,3915837,127094918,1629589924,18860,13071725,131010755,17516.5,586216.4,603732.9,0,6199863875.83601522968887535217342
2021-08-22 23:59:52,3944736.3,128042820.3,1629676792,18861,13078270,131987556.6,28899.3,947902.3,976801.6,0,6209714982.109681277349913884682856


In [44]:
def df_to_numeric(df):
    for column in df:
        try:
            df[column] = pd.to_numeric(df[column])
        except Exception as e:
            pass

In [45]:
df_to_numeric(df)

In [48]:
# Graphing the data
fig = px.line(df, x = df.index, y=['cumulativeProtocolSideRevenueUSD', 'cumulativeSupplySideRevenueUSD',
       'cumulativeTotalRevenueUSD', 'dailyProtocolSideRevenueUSD', 'dailySupplySideRevenueUSD',
       'dailyTotalRevenueUSD', 'protocolControlledValueUSD', 'totalValueLockedUSD'],
    height=500, title = "Convex Finance Revenues")
fig.update_traces(mode="lines", hovertemplate=None)
fig.update_layout(hovermode="x") #, yaxis= {"tickformat": ',.2%' })
fig.show() 

In [ ]:
import pandas as pd

url = "https://github.com/credprotocol/Aave-V2-Health-Factor-Dataset/blob/main/data/0x01acb3804ba9c42111c6e9c127831eb486ca1ac7.csv?raw=True"
df = pd.read_csv(url)

In [ ]:
list(df.columns)

In [ ]:
df